In [1]:
%%capture
!pip install langchain==0.1.4 openai==1.10.0 langchain-openai

In [11]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# Prompt Pipelining Overview


- 🧱 **Modularity**: Mix and match prompt pieces like LEGO blocks.
- 👓 **Readability**: Break down complex prompts into easy bits.
- 🧠**Flexibility**: Craft prompts on-the-go with logic-based assembly.
- 🔄 **Efficiency**: Loop to append for scenarios like few-shot learning.
- 📝✨**Hybrid Construction**: Combine fixed text with variable-filled templates for structure and spontaneity.
- 💬 **Chat-Friendly**: Create conversational prompts by stacking messages.
- 🛠️ **Customizability**: Let users build prompts with their own components.

**String Prompt Pipelining:**

- 🔗 **Sequential Flow**: Link templates or strings in order, starting with a prompt.

Prompt pipelining turns the art of prompt crafting into a modular, efficient process, perfect for those looking to streamline and enhance their prompt design. 💡


In [12]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("I'm heading to {destination}. ")
    + "Recommend a great {activity} spot!")

prompt

PromptTemplate(input_variables=['activity', 'destination'], template="I'm heading to {destination}. Recommend a great {activity} spot!")

In [13]:
prompt = prompt + "\n\nAlso, any local delicacies I should try?"
prompt

PromptTemplate(input_variables=['activity', 'destination'], template="I'm heading to {destination}. Recommend a great {activity} spot!\n\nAlso, any local delicacies I should try?")

In [14]:
print(prompt.template)

I'm heading to {destination}. Recommend a great {activity} spot!

Also, any local delicacies I should try?


In [15]:
prompt.format(destination="Punjab", activity="dining")

"I'm heading to Punjab. Recommend a great dining spot!\n\nAlso, any local delicacies I should try?"

### The key differences between prompt pipelining and multi-input prompts are:

🚂 **Composition**: Pipelining links individual prompts into a cohesive journey.

🖼️ **Independence**: Each pipeline component is crafted separately before integration.

🚶‍♂️ **Sequence**: Pipelining lines up components, unlike multi-input prompts that handle inputs collectively.

📋 **Reuse**: Pipelining excels in reusing pieces; multi-input prompts manage multiple data points in one go.

📖 **Outcome**: Pipelining produces a single narrative; multi-input prompts generate a combined result.

🧱 **Construction**: Pipelining is about assembling prompts step by step, while multi-input prompts are about managing various inputs at once.

In short, pipelining is like creating a melody note by note, whereas multi-input prompts are like playing chords, hitting multiple notes simultaneously.


# Use in a chain

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [17]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

for chunk in chain.stream({"destination":"Punjab", "activity":"dining"}):
  print(chunk, end="", flush=True)

One of the popular dining spots in Punjab is "Sarson Da Saag and Makki Di Roti" which is known for its traditional Punjabi cuisine. Another great option is "Kesar Da Dhaba" known for its delicious vegetarian dishes.

When in Punjab, you must try the local delicacies such as Butter Chicken, Tandoori Chicken, Amritsari Fish, Chole Bhature, and Lassi. These dishes are a must-try for anyone visiting Punjab.

In [18]:
prompt = prompt + " How should I greet the locals in a jolly, informal manner?"


In [19]:
prompt.template

"I'm heading to {destination}. Recommend a great {activity} spot!\n\nAlso, any local delicacies I should try? How should I greet the locals in a jolly, informal manner?"

In [20]:
chain = prompt | llm | output_parser

for chunk in chain.stream({"destination":"Punjab", "activity":"dining"}):
  print(chunk, end="", flush=True)

One great dining spot in Punjab is the famous Kesar Da Dhaba in Amritsar, known for its delicious Punjabi cuisine and traditional ambiance. 

Some local delicacies you should try include sarson da saag and makki di roti, chole bhature, butter chicken, and lassi.

To greet the locals in a jolly, informal manner, you can use "Sat Sri Akal" or "Sasriakal", which is a common Punjabi greeting. It translates to "may you have a long life" and is often used to say hello.

# Example usecase

In [21]:
class TravelChatbot:
    def __init__(self, base_template):
        self.model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)
        self.base_prompt = PromptTemplate.from_template(base_template)

    def append_to_prompt(self, additional_text):
        self.base_prompt += additional_text

    def run_chain(self, destination, activity):
        output_parser = StrOutputParser()
        chain = self.base_prompt | self.model | output_parser
        for chunk in chain.stream({"destination":destination, "activity":activity}):
          print(chunk, end="", flush=True)

# Usage
base_template = "I'm heading to {destination}. Recommend a great {activity} spot!"

chatbot = TravelChatbot(base_template)

# Basic prompt
chatbot.run_chain(destination="Punjab", activity="dining")

Sure! One highly recommended dining spot in Punjab is Barbeque Nation. They offer a wide variety of delicious barbeque dishes and a great ambiance for a memorable dining experience. Another popular option is Kesar da Dhaba, known for its authentic Punjabi cuisine and traditional atmosphere. Both of these restaurants are sure to provide a fantastic dining experience during your visit to Punjab.

In [22]:
# Append more to the prompt and run again
chatbot.append_to_prompt("\n\nAlso, any local delicacies I should try?")

chatbot.run_chain(destination="Punjab", activity="dining")

One of the best dining spots in Punjab is Bukhara Grill, known for its delicious tandoori dishes and traditional Punjabi cuisine. Another popular choice is Kesar da Dhaba, where you can savor authentic Punjabi flavors in a rustic setting.

As for local delicacies, be sure to try the famous Amritsari Kulcha, a type of Indian bread stuffed with spiced potatoes and served with chole (chickpea curry). Also, don't miss out on the rich and creamy Sarson da Saag (mustard greens curry) served with Makki di Roti (corn bread). And of course, indulge in some delicious Punjabi sweets like Gulab Jamun and Jalebi. Enjoy your culinary adventure in Punjab!

In [23]:
chatbot.append_to_prompt(" How should I greet the locals in a friendly, informal, jolly colloquial manner?")

chatbot.run_chain(destination="Punjab", activity="dining")

One great dining spot in Punjab is Makhan Fish & Chicken Corner in Amritsar, known for its delicious Punjabi cuisine and famous fish and chicken dishes.

Some local delicacies you should try include butter chicken, tandoori chicken, saag and makki di roti, chole bhature, and lassi.

To greet the locals in a friendly, informal, jolly colloquial manner, you can say "Sat Sri Akal" or "Sasriakal" which means "Hello" in Punjabi. It's sure to bring a smile to their faces!

# Chat Prompt Pipeline

🧩 **Composition**: Chat prompt pipelining turns reusable message blocks into a complete conversation flow.

🛠️ **Versatility**: Mix and match static messages with dynamic templates for a custom dialogue.

🔗 **End Result**: You get a ChatPromptTemplate that's ready for action, crafted from your message lineup.

🏗️ **Modularity**: Like using building blocks, this method lets you construct prompts piece by piece for maximum flexibility.

In essence, chat prompt pipelining is about assembling conversations from logical blocks, creating a user-friendly and adaptable ChatPromptTemplate.


In [24]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Setting the scene with a Cockney-themed system message
prompt = SystemMessage(content="Welcome to the East End Cockney Chat! 🇬🇧")

# Constructing a chat flow with dry humour
new_prompt = (
    prompt
    + HumanMessage(content="Alright, guv'nor?")
    + AIMessage(content="Not too shabby. Did you hear about the London fog?")
    + "{input}"
)

# Formatting the chat with the user's response
new_prompt.format_messages(input="No, what about it?")

print(new_prompt)


input_variables=['input'] messages=[SystemMessage(content='Welcome to the East End Cockney Chat! 🇬🇧'), HumanMessage(content="Alright, guv'nor?"), AIMessage(content='Not too shabby. Did you hear about the London fog?'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


In [25]:
new_prompt.messages

[SystemMessage(content='Welcome to the East End Cockney Chat! 🇬🇧'),
 HumanMessage(content="Alright, guv'nor?"),
 AIMessage(content='Not too shabby. Did you hear about the London fog?'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]

In [26]:
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

output_parser = StrOutputParser()

chain = new_prompt | model | output_parser

# Running the chatbot to get the punchline
for chunk in chain.stream({"input":"No, what about it?"}):
  print(chunk, end="", flush=True)

It was pea souper, mate! Couldn't see me hand in front of me face!

# Prompt Composition


🧬 **Prompt Composition**: Reuse prompt segments with ease using the PipelinePrompt feature.

🏁 **1. Final Prompt**: The end product that you present to the model.

🔗 **2. Pipeline Prompts**: A sequence of named prompt templates that pass information forward, each influencing the next.

To summarize, PipelinePrompt allows for the efficient building of complex prompts by reusing and chaining together smaller, named components.


In [27]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [28]:
full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

In [29]:
full_prompt.input_variables

['example', 'introduction', 'start']

In [30]:
introduction_template = """You are impersonating {person}."""

introduction_prompt = PromptTemplate.from_template(introduction_template)

In [31]:
introduction_prompt.input_variables

['person']

In [32]:
example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""

example_prompt = PromptTemplate.from_template(example_template)

In [33]:
start_template = """Now, do this for real!

Q: {input}
A:"""

start_prompt = PromptTemplate.from_template(start_template)

In [34]:
full_prompt

PromptTemplate(input_variables=['example', 'introduction', 'start'], template='{introduction}\n\n{example}\n\n{start}')

In [35]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [36]:
pipeline_prompt.input_variables

['example_a', 'example_q', 'person', 'input']

In [37]:
last_prompt = pipeline_prompt.format(
    person="Elon Musk",
    example_q="What's your favorite car?",
    example_a="Tesla",
    input="What's your favorite social media site?"
)

print(last_prompt)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


In [38]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.75)

chain = pipeline_prompt | llm | StrOutputParser()

chain.invoke({
    "person":"Elon Musk","example_q":"What's your favorite car?",
    "example_a":"Tesla",
    "input":"What's your favorite social media site and why?"
    })

'Twitter, because it allows me to directly communicate with my followers and share updates and news about my companies and projects. Plus, the character limit forces me to be concise and to the point.'

In [39]:
from langchain.prompts import PromptTemplate, PipelinePromptTemplate

class CookingShowChatbot:
    def __init__(self):
        # Base template for the cooking show scenario
        self.full_template = """
        {introduction}

        {example_dish}

        {present_dish}"""
        self.full_prompt = PromptTemplate.from_template(self.full_template)

        # Introduction where the user impersonates a famous chef
        self.introduction_template = """Welcome to the cooking show! Today, you're channeling the spirit of Chef {chef_name}."""
        self.introduction_prompt = PromptTemplate.from_template(self.introduction_template)

        # Example dish made by the famous chef
        self.example_dish_template = """Remember when Chef {chef_name} made that delicious {example_dish_name}? It was a hit!"""
        self.example_dish_prompt = PromptTemplate.from_template(self.example_dish_template)

        # User's turn to present their dish
        self.present_dish_template = """Now, it's your turn! Show us how you make your {user_dish_name}. Let's get cooking!"""
        self.present_dish_prompt = PromptTemplate.from_template(self.present_dish_template)

        # Combining the prompts into a pipeline
        self.input_prompts = [
            ("introduction", self.introduction_prompt),
            ("example_dish", self.example_dish_prompt),
            ("present_dish", self.present_dish_prompt)
        ]
        self.pipeline_prompt = PipelinePromptTemplate(final_prompt=self.full_prompt,
                                                      pipeline_prompts=self.input_prompts
                                                      )

    def run_scenario(self, chef_name, example_dish_name, user_dish_name):
        chain = self.pipeline_prompt | llm | StrOutputParser()

        response = chain.invoke({"chef_name":chef_name, "example_dish_name":example_dish_name, "user_dish_name":user_dish_name})

        return response




In [40]:
chatbot = CookingShowChatbot()
scenario = chatbot.run_scenario(chef_name="Gordon Ramsay", example_dish_name="Beef Wellington", user_dish_name="Vegan Lasagna")
print(scenario)



Alright, let's get started on this vegan lasagna. First, preheat the oven to 375°F (190°C). 

Next, we'll need some lasagna noodles, marinara sauce, tofu ricotta, spinach, mushrooms, and vegan mozzarella cheese.

Start by cooking the lasagna noodles according to the package instructions. While that's cooking, sauté the mushrooms and spinach in a pan with a little olive oil until they're softened.

Now, it's time to assemble the lasagna. Start by spreading a layer of marinara sauce on the bottom of a baking dish. Then add a layer of the cooked lasagna noodles, followed by a layer of the tofu ricotta, sautéed mushrooms and spinach, and a sprinkle of vegan mozzarella cheese. Repeat these layers until you've used up all your ingredients, finishing with a layer of marinara sauce and vegan mozzarella cheese on top.

Cover the dish with foil and bake in the preheated oven for 30 minutes. Then, remove the foil and bake for an additional 15 minutes, or until the cheese is melted and bubbly.

